<a href="https://colab.research.google.com/github/hoky1227/Transformer_based-recommendation/blob/main/TransformRec_e_commerce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is TransformRec sample code using e-commerce dataset. 9/10/2021.

# Gdrive

In [1]:
from google.colab import drive

drive.mount('/content/drive')

path = '/content/drive/MyDrive/'

Mounted at /content/drive


# Import Modules

In [2]:
import numpy as np
import pandas as pd

from dateutil.parser import parse
from tqdm import tqdm_notebook
import tqdm

import tensorflow_datasets as tfds
import tensorflow as tf
import re

from gensim.models import Word2Vec
import math

import warnings 

warnings.filterwarnings('ignore')


np.random.seed(2021)
tf.random.set_seed(2021)

In [3]:
import dateutil
import gensim

print(np.__version__)
print(pd.__version__)
print(dateutil.__version__)
print(tfds.__version__)
print(tf.__version__)
print(gensim.__version__)

1.19.5
1.1.5
2.8.2
4.0.1
2.6.0
3.6.0


# Data Preprocessing

In [4]:
df = pd.read_csv("/content/drive/MyDrive/2017online retail.csv")
print(len(df))
display(df.describe())
print(df.isnull().sum())
df.dropna(inplace=True)
print(df.duplicated().sum())
df.drop_duplicates(inplace=True)
print(len(df))
df.head()

210367


,Customer,Amount,Price
count,167397.000000,210367.000000,210367.000000
mean,35318.776119,10.650625,6.981082
std,1717.755324,121.362892,123.976776
min,15261.000000,-9200.000000,-3594.360000
25%,33971.000000,1.000000,1.250000
50%,35227.000000,3.000000,2.100000
75%,36798.000000,10.000000,4.250000
max,49001.000000,19152.000000,10953.500000


Customer       42970
Payment Way        0
Invoice            0
ItemCode           0
Detail          1745
Date               0
Amount             0
Price              0
dtype: int64
1458
165939


,Customer,Payment Way,Invoice,ItemCode,Detail,Date,Amount,Price
0,37593.0,Skrill,C514216,22406,MONEY BOX KINGS CHOICE DESIGN,2017/6/30 17:38,-1,1.25
1,37593.0,Skrill,C514216,22056,CERAMIC PLATE LOVE HEART DESIGN,2017/6/30 17:38,-1,1.49
2,37593.0,Skrill,C514216,21844,RED RETROSPOT MUG,2017/6/30 17:38,-1,2.95
3,37593.0,Skrill,C514216,22073,RED RETROSPOT STORAGE JAR,2017/6/30 17:38,-1,3.75
4,37593.0,Credit,C514216,22059,CERAMIC STRWBERRY DESIGN MUG,2017/6/30 17:38,-2,1.49


In [5]:
print(len(df['Customer'].unique()))
print(len(df['Invoice'].unique()))
print(len(df['Detail'].unique()))

2919
9656
3549


In [6]:
df.drop(['Payment Way', 'Date'],axis=1, inplace=True)
df.head()

,Customer,Invoice,ItemCode,Detail,Amount,Price
0,37593.0,C514216,22406,MONEY BOX KINGS CHOICE DESIGN,-1,1.25
1,37593.0,C514216,22056,CERAMIC PLATE LOVE HEART DESIGN,-1,1.49
2,37593.0,C514216,21844,RED RETROSPOT MUG,-1,2.95
3,37593.0,C514216,22073,RED RETROSPOT STORAGE JAR,-1,3.75
4,37593.0,C514216,22059,CERAMIC STRWBERRY DESIGN MUG,-2,1.49


In [7]:
def check():
    df['Detail'] = df['Detail'].str.strip()
    Detail = df['Detail'].unique()
    for i in Detail:
        if i.isupper() == False:
            print(i)

check()

Manual
BAG 250g SWIRLY MARBLES
Discount
Adjustment by Peter on 24/05/2010 1
BAG 125g SWIRLY MARBLES
POLYESTER FILLER PAD 65CMx65CM
Bank Charges
POLYESTER FILLER PAD 40x40cm
Adjustment by john on 26/01/2010 17
Adjustment by john on 26/01/2010 16
Adjust bad debt
FOLK ART GREETING CARD,pack/12
POLYESTER FILLER PAD 45x45cm
Adjustment by Peter on Jun 25 2010
ESSENTIAL BALM 3.5g TIN IN ENVELOPE
SET Of 6 SOLDIER SKITTLES
POLYESTER FILLER PAD 30CMx30CM
THE KING GIFT BAG 25x24x12cm
POLYESTER FILLER PAD 45x30cm
This is a test product.
POLYESTER FILLER PAD 60x40cm


In [8]:
def check10():
    Detail = df['Detail'].unique()
    for i in Detail:
        if len(i) <= 10:
            print(i)

check10()

Manual
POSTAGE
Discount
CARRIAGE
SPOTS MUG
PHOTO CUBE
SOMBRERO
BINGO SET
SPACE OWL
CAT BOWL
SPACE FROG
DAISY RING


In [9]:
idx = df[df['Amount'] <= 0.].index
df.drop(idx, inplace=True)
l = ['Manual', 'This is a test product.', 'Discount', 'Adjustment by Peter on 24/05/2010 1',\
     'Bank Charges', 'Adjustment by john on 26/01/2010 17',  'Adjustment by john on 26/01/2010 16', 'Adjust bad debt', 'Adjustment by Peter on Jun 25 2010', 'POSTAGE', 'CARRIAGE']
for v in l:
    idx2 = df[df['Detail'] == v].index
    df.drop(idx2, inplace=True)
idx3 = df[df['Price'] <= 0.].index
df.drop(idx3, inplace=True)

df.describe()

,Customer,Amount,Price
count,161040.000000,161040.000000,161040.000000
mean,35334.207570,14.045293,3.051039
std,1673.922109,123.241911,3.435328
min,32346.000000,1.000000,0.001000
25%,33976.000000,2.000000,1.250000
50%,35248.000000,5.000000,1.950000
75%,36805.000000,12.000000,3.750000
max,38287.000000,19152.000000,125.000000


In [10]:
check()

BAG 250g SWIRLY MARBLES
FOLK ART GREETING CARD,pack/12
POLYESTER FILLER PAD 45x45cm
POLYESTER FILLER PAD 40x40cm
BAG 125g SWIRLY MARBLES
POLYESTER FILLER PAD 65CMx65CM
ESSENTIAL BALM 3.5g TIN IN ENVELOPE
SET Of 6 SOLDIER SKITTLES
POLYESTER FILLER PAD 30CMx30CM
THE KING GIFT BAG 25x24x12cm
POLYESTER FILLER PAD 45x30cm
POLYESTER FILLER PAD 60x40cm


In [11]:
check10()

PHOTO CUBE
SOMBRERO
BINGO SET
SPACE OWL
CAT BOWL
SPOTS MUG
SPACE FROG
DAISY RING


In [12]:
df.drop(['Amount', 'Price', 'ItemCode'],axis=1, inplace=True)
df.head()

,Customer,Invoice,Detail
4492,34882.0,264212,COLOUR GLASS T-LIGHT HOLDER HANGING
4493,34882.0,264212,PORCELAIN HANGING BELL SMALL
4494,34882.0,264212,NATURAL SLATE HEART CHALKBOARD
4496,34882.0,264212,PORCELAIN HANGING BELL LARGE
4497,34882.0,264212,ENAMEL WATERING CAN CREAM


In [13]:
detail = df['Detail'].unique()

In [14]:
def sortdf(data, column):
    data.sort_values(column, inplace=True)
    data.reset_index(inplace=True)
    data.pop('index')
    return data

In [15]:
df['Detail'].value_counts()

WHITE HANGING HEART T-LIGHT HOLDER    1385
PACK OF 72 RETRO SPOT CAKE CASES       693
DOOR MAT UNION FLAG                    632
LUNCH BAG RED SPOTTY                   602
REGENCY CAKESTAND 3 TIER               594
                                      ... 
RED   ACRYLIC FACETED BANGLE             1
ASSORTED TUTTI FRUTTI KEYRING            1
RED STONE/CRYSTAL EARRINGS               1
BAROQUE BUTTERFLY EARRINGS CRYSTAL       1
AMBER BERTIE MOBILE PHONE CHARM          1
Name: Detail, Length: 3485, dtype: int64

In [16]:
len(df)

161040

In [17]:
df['Detail'].value_counts().describe()

count    3485.000000
mean       46.209469
std        74.823246
min         1.000000
25%         5.000000
50%        17.000000
75%        56.000000
max      1385.000000
Name: Detail, dtype: float64

In [18]:
df['Detail'].value_counts()[:10].index.to_list()

['WHITE HANGING HEART T-LIGHT HOLDER',
 'PACK OF 72 RETRO SPOT CAKE CASES',
 'DOOR MAT UNION FLAG',
 'LUNCH BAG RED SPOTTY',
 'REGENCY CAKESTAND 3 TIER',
 'STRAWBERRY CERAMIC TRINKET BOX',
 'HOME BUILDING BLOCK WORD',
 '60 TEATIME FAIRY CAKE CASES',
 'ASSORTED COLOUR BIRD ORNAMENT',
 'SET/20 RED SPOTTY PAPER NAPKINS']

## Invoice based history

In [19]:
df_invoice = df.copy()
df_invoice['Invoice'] = df_invoice['Invoice'].apply(lambda x: str(x))
sortdf(df_invoice, ['Invoice'])
df_invoice.head()

,Customer,Invoice,Detail
0,34590.0,243414,HAND OPEN SHAPE GOLD
1,34590.0,243414,RETRO SPOT MUG
2,34590.0,243414,RETRO SPOT SUGAR JAM BOWL
3,34590.0,243414,NEW ENGLAND CERAMIC CAKE SERVER
4,34590.0,243414,RETRO SPOT TRADITIONAL TEAPOT


In [20]:
Invoice = df_invoice['Invoice'].unique()
Invoice = sorted(Invoice)
len(Invoice)

7557

In [21]:
df3 = pd.DataFrame(columns=['Invoice', 'Detail'])
values = df_invoice['Invoice'].value_counts().index.sort_values()

for i, v in enumerate(tqdm.tqdm(values)):
    df3.loc[i] = (v, '&&'.join(df_invoice[df_invoice['Invoice'] == v]['Detail']))
df3

100%|██████████| 7557/7557 [01:29<00:00, 84.03it/s]


,Invoice,Detail
0,243414,HAND OPEN SHAPE GOLD&&RETRO SPOT MUG&&RETRO SP...
1,243427,MOROCCAN TEA GLASS&&BLACK FLOWER CANDLE PLATE&...
2,243428,LUNCHBOX WITH CUTLERY FAIRY CAKES&&STRAWBERRY ...
3,243432,RED WOOLLY HOTTIE WHITE HEART.&&CERAMIC CAKE D...
4,243433,STRAWBERRY CERAMIC TRINKET BOX&&JUMBO BAG BAR...
...,...,...
7552,264208,ASSORTED COLOUR BIRD ORNAMENT&&VICTORIAN GLASS...
7553,264209,PACK OF 60 PINK PAISLEY CAKE CASES&&WASHROOM M...
7554,264210,GLASS JAR DIGESTIVE BISCUITS&&BLUE STRIPE CERA...
7555,264211,PICNIC BASKET WICKER SMALL&&NATURAL SLATE HEAR...


In [22]:
df3.pop('Invoice')
df3

,Detail
0,HAND OPEN SHAPE GOLD&&RETRO SPOT MUG&&RETRO SP...
1,MOROCCAN TEA GLASS&&BLACK FLOWER CANDLE PLATE&...
2,LUNCHBOX WITH CUTLERY FAIRY CAKES&&STRAWBERRY ...
3,RED WOOLLY HOTTIE WHITE HEART.&&CERAMIC CAKE D...
4,STRAWBERRY CERAMIC TRINKET BOX&&JUMBO BAG BAR...
...,...
7552,ASSORTED COLOUR BIRD ORNAMENT&&VICTORIAN GLASS...
7553,PACK OF 60 PINK PAISLEY CAKE CASES&&WASHROOM M...
7554,GLASS JAR DIGESTIVE BISCUITS&&BLUE STRIPE CERA...
7555,PICNIC BASKET WICKER SMALL&&NATURAL SLATE HEAR...


In [23]:
df_series = df3['Detail'].str.split('&&')
df_series[0]

['HAND OPEN SHAPE GOLD',
 'RETRO SPOT MUG',
 'RETRO SPOT SUGAR JAM BOWL',
 'NEW ENGLAND CERAMIC CAKE SERVER',
 'RETRO SPOT TRADITIONAL TEAPOT',
 'RETRO SPOT LARGE MILK JUG']

In [24]:
sequence_length = 5
step_size = 2

def create_sequences(values, window_size, step_size):
    sequences = []
    start_index = 0
    while True:
        end_index = start_index + window_size
        seq = values[start_index:end_index]

        if len(seq) < window_size:
            seq = values[-window_size:]

            if len(seq) == window_size:
                sequences.append(seq)
            break
        sequences.append(seq)
        start_index += step_size
    return sequences

In [25]:
for i, v in enumerate(df_series):
    df_series[i] = create_sequences(v, sequence_length, step_size)
df_series[0]

[['HAND OPEN SHAPE GOLD',
  'RETRO SPOT MUG',
  'RETRO SPOT SUGAR JAM BOWL',
  'NEW ENGLAND CERAMIC CAKE SERVER',
  'RETRO SPOT TRADITIONAL TEAPOT'],
 ['RETRO SPOT MUG',
  'RETRO SPOT SUGAR JAM BOWL',
  'NEW ENGLAND CERAMIC CAKE SERVER',
  'RETRO SPOT TRADITIONAL TEAPOT',
  'RETRO SPOT LARGE MILK JUG']]

In [26]:
df_list = []
for v in df_series:
    for value in v:
        df_list.append(value)
df_list[:2]

[['HAND OPEN SHAPE GOLD',
  'RETRO SPOT MUG',
  'RETRO SPOT SUGAR JAM BOWL',
  'NEW ENGLAND CERAMIC CAKE SERVER',
  'RETRO SPOT TRADITIONAL TEAPOT'],
 ['RETRO SPOT MUG',
  'RETRO SPOT SUGAR JAM BOWL',
  'NEW ENGLAND CERAMIC CAKE SERVER',
  'RETRO SPOT TRADITIONAL TEAPOT',
  'RETRO SPOT LARGE MILK JUG']]

In [27]:
label_list = []

for i in range(len(df_list)):
    l = df_list[i][-1]
    df_list[i].pop()
    label_list.append(l)
label_list[:2]

['RETRO SPOT TRADITIONAL TEAPOT', 'RETRO SPOT LARGE MILK JUG']

In [28]:
df_new = pd.DataFrame(columns=['Detail', 'label'])
df_new['Detail'] = df_list
df_new['label'] = label_list
df_new

,Detail,label
0,"[HAND OPEN SHAPE GOLD, RETRO SPOT MUG, RETRO S...",RETRO SPOT TRADITIONAL TEAPOT
1,"[RETRO SPOT MUG, RETRO SPOT SUGAR JAM BOWL, NE...",RETRO SPOT LARGE MILK JUG
2,"[MOROCCAN TEA GLASS, BLACK FLOWER CANDLE PLATE...",CERAMIC CAKE DESIGN SPOTTED PLATE
3,"[BLACK SILOUETTE CANDLE PLATE, VICTORIAN GLASS...",SAVE THE PLANET COTTON TOTE BAG
4,"[CERAMIC CAKE DESIGN SPOTTED PLATE, PINK CLEAR...",STRAWBERRY CERAMIC TRINKET BOX
...,...,...
74502,"[UNION STRIPE WITH FRINGE HAMMOCK, RED/CREAM ...",WHITE HANGING HEART T-LIGHT HOLDER
74503,"[OCEAN STRIPE HAMMOCK, BLUE/CREAM STRIPE FRING...",ENAMEL WATERING CAN CREAM
74504,"[WHITE HANGING HEART T-LIGHT HOLDER, BAG 250g ...",NATURAL SLATE HEART CHALKBOARD
74505,"[ENAMEL WATERING CAN CREAM, PORCELAIN HANGING ...",COLOUR GLASS T-LIGHT HOLDER HANGING


In [29]:
df_new['Detail'] = df_new['Detail'].apply(lambda x: '&&'.join(map(str, x)))
df_new

,Detail,label
0,HAND OPEN SHAPE GOLD&&RETRO SPOT MUG&&RETRO SP...,RETRO SPOT TRADITIONAL TEAPOT
1,RETRO SPOT MUG&&RETRO SPOT SUGAR JAM BOWL&&NEW...,RETRO SPOT LARGE MILK JUG
2,MOROCCAN TEA GLASS&&BLACK FLOWER CANDLE PLATE&...,CERAMIC CAKE DESIGN SPOTTED PLATE
3,BLACK SILOUETTE CANDLE PLATE&&VICTORIAN GLASS ...,SAVE THE PLANET COTTON TOTE BAG
4,CERAMIC CAKE DESIGN SPOTTED PLATE&&PINK CLEAR ...,STRAWBERRY CERAMIC TRINKET BOX
...,...,...
74502,UNION STRIPE WITH FRINGE HAMMOCK&&RED/CREAM S...,WHITE HANGING HEART T-LIGHT HOLDER
74503,OCEAN STRIPE HAMMOCK&&BLUE/CREAM STRIPE FRINGE...,ENAMEL WATERING CAN CREAM
74504,WHITE HANGING HEART T-LIGHT HOLDER&&BAG 250g S...,NATURAL SLATE HEART CHALKBOARD
74505,ENAMEL WATERING CAN CREAM&&PORCELAIN HANGING B...,COLOUR GLASS T-LIGHT HOLDER HANGING


In [30]:
print(df_new.isnull().sum())
df_new.dropna(inplace=True)

Detail    0
label     0
dtype: int64


In [31]:
print(df_new.duplicated().sum())
df_new.drop_duplicates(inplace=True)
df_new.reset_index(inplace=True)
df_new.pop('index')
df_new

3302


,Detail,label
0,HAND OPEN SHAPE GOLD&&RETRO SPOT MUG&&RETRO SP...,RETRO SPOT TRADITIONAL TEAPOT
1,RETRO SPOT MUG&&RETRO SPOT SUGAR JAM BOWL&&NEW...,RETRO SPOT LARGE MILK JUG
2,MOROCCAN TEA GLASS&&BLACK FLOWER CANDLE PLATE&...,CERAMIC CAKE DESIGN SPOTTED PLATE
3,BLACK SILOUETTE CANDLE PLATE&&VICTORIAN GLASS ...,SAVE THE PLANET COTTON TOTE BAG
4,CERAMIC CAKE DESIGN SPOTTED PLATE&&PINK CLEAR ...,STRAWBERRY CERAMIC TRINKET BOX
...,...,...
71200,WOODEN SKITTLES GARDEN SET&&DOORMAT UNION FLAG...,WOODEN CROQUET GARDEN SET
71201,UNION STRIPE WITH FRINGE HAMMOCK&&RED/CREAM S...,WHITE HANGING HEART T-LIGHT HOLDER
71202,OCEAN STRIPE HAMMOCK&&BLUE/CREAM STRIPE FRINGE...,ENAMEL WATERING CAN CREAM
71203,WHITE HANGING HEART T-LIGHT HOLDER&&BAG 250g S...,NATURAL SLATE HEART CHALKBOARD


In [32]:
ds_invoice = df_new

# Transformer

In [33]:
data = ds_invoice

data = data.sample(frac=1).reset_index(drop=True)
data.head()

,Detail,label
0,JUMBO BAG RED WHITE SPOTTY&&JUMBO BAG PINK WIT...,LUNCH BAG PINK RETROSPOT
1,PLEASE ONE PERSON METAL SIGN&&NATURAL SLATE HE...,NATURAL SLATE CHALKBOARD LARGE
2,JUMBO BAG WOODLAND ANIMALS&&STRAWBERRY CANDY B...,WOODLAND CHARLOTTE BAG
3,PINK DOUGHNUT TRINKET POT&&VINTAGE HEADS AND T...,COFFEE MUG BLUE PAISLEY DESIGN
4,3D HEARTS HONEYCOMB PAPER GARLAND&&SPOTTY HO...,ORIENTAL BLUE C/COVER


In [34]:
train_df = data.copy()
train_df.rename(columns = {'Detail':'train'}, inplace=True)
train_df.head()

,train,label
0,JUMBO BAG RED WHITE SPOTTY&&JUMBO BAG PINK WIT...,LUNCH BAG PINK RETROSPOT
1,PLEASE ONE PERSON METAL SIGN&&NATURAL SLATE HE...,NATURAL SLATE CHALKBOARD LARGE
2,JUMBO BAG WOODLAND ANIMALS&&STRAWBERRY CANDY B...,WOODLAND CHARLOTTE BAG
3,PINK DOUGHNUT TRINKET POT&&VINTAGE HEADS AND T...,COFFEE MUG BLUE PAISLEY DESIGN
4,3D HEARTS HONEYCOMB PAPER GARLAND&&SPOTTY HO...,ORIENTAL BLUE C/COVER


In [35]:
filtering_train_df = train_df.copy()

In [36]:
# train : test = 8 : 2
cut = round(len(filtering_train_df) * 0.8)
filtering_train_df, filtering_test_df = filtering_train_df[:cut], filtering_train_df[cut:]

In [37]:
filtering_test_df = filtering_test_df.reset_index(drop=True)

In [38]:
print(len(filtering_train_df))
len(filtering_test_df)

56964


14241

In [39]:
filtering_train_df.to_csv(path + 'e-commerce_train.csv')
filtering_test_df.to_csv(path + 'e-commerce_test.csv')

In [40]:
filtering_train_df['split_train_token'] = ['<esp> ' + i.strip().replace('&&', ' <esp> ') + ' <esp>' for i in filtering_train_df['train']]
filtering_train_df['split_label_token'] = ['<esp> ' + i.strip() + ' <esp>' for i in filtering_train_df['label']]

In [41]:
filtering_train_df.head()

,train,label,split_train_token,split_label_token
0,JUMBO BAG RED WHITE SPOTTY&&JUMBO BAG PINK WIT...,LUNCH BAG PINK RETROSPOT,<esp> JUMBO BAG RED WHITE SPOTTY <esp> JUMBO B...,<esp> LUNCH BAG PINK RETROSPOT <esp>
1,PLEASE ONE PERSON METAL SIGN&&NATURAL SLATE HE...,NATURAL SLATE CHALKBOARD LARGE,<esp> PLEASE ONE PERSON METAL SIGN <esp> NATUR...,<esp> NATURAL SLATE CHALKBOARD LARGE <esp>
2,JUMBO BAG WOODLAND ANIMALS&&STRAWBERRY CANDY B...,WOODLAND CHARLOTTE BAG,<esp> JUMBO BAG WOODLAND ANIMALS <esp> STRAWBE...,<esp> WOODLAND CHARLOTTE BAG <esp>
3,PINK DOUGHNUT TRINKET POT&&VINTAGE HEADS AND T...,COFFEE MUG BLUE PAISLEY DESIGN,<esp> PINK DOUGHNUT TRINKET POT <esp> VINTAGE ...,<esp> COFFEE MUG BLUE PAISLEY DESIGN <esp>
4,3D HEARTS HONEYCOMB PAPER GARLAND&&SPOTTY HO...,ORIENTAL BLUE C/COVER,<esp> 3D HEARTS HONEYCOMB PAPER GARLAND <esp>...,<esp> ORIENTAL BLUE C/COVER <esp>


In [42]:
MAX_LENGTH = 128

In [43]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(filtering_train_df['split_train_token'] + filtering_train_df['split_label_token'], target_vocab_size=2**13)

In [44]:
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
VOCAB_SIZE = tokenizer.vocab_size + 2

In [45]:
print(START_TOKEN)
print(END_TOKEN)
print(VOCAB_SIZE)

[2717]
[2718]
2719


In [46]:
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []

  for (sentence1, sentence2) in zip(inputs, outputs):
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    tokenized_inputs.append(sentence1)
    tokenized_outputs.append(sentence2)

  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')

  return tokenized_inputs, tokenized_outputs

In [47]:
train_token, label_token = tokenize_and_filter(filtering_train_df['split_train_token'], filtering_train_df['split_label_token'])

In [48]:
print(train_token.shape)
print(label_token.shape)

(56964, 128)
(56964, 128)


In [49]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [50]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [51]:
!nvidia-smi # Using K80 (Colab) in this example.

Fri Sep 10 10:48:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    58W / 149W |    121MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [52]:
BATCH_SIZE = 256
BUFFER_SIZE = 20000

dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': train_token,
        'dec_inputs': label_token[:, :-1]
    },
    {
        'outputs': label_token[:, 1:]
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [53]:
def scaled_dot_product_attention(query, key, value, mask):
  """Calculate the attention weights. """
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  if mask is not None:
    logits += (mask * -1e9)

  attention_weights = tf.nn.softmax(logits, axis=-1)

  output = tf.matmul(attention_weights, value)

  return output


class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    outputs = self.dense(concat_attention)

    return outputs


def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
  return mask[:, tf.newaxis, tf.newaxis, :]

def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)


class PositionalEncoding(tf.keras.layers.Layer):

  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)

    sines = tf.math.sin(angle_rads[:, 0::2])

    cosines = tf.math.cos(angle_rads[:, 1::2])

    pos_encoding = tf.concat([sines, cosines], axis=-1)
    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)
  
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)
  
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)
  
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = decoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  enc_outputs = encoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[inputs, enc_padding_mask])

  dec_outputs = decoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [54]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  
  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

In [55]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [56]:
tf.keras.backend.clear_session()

# Hyperparameters

# NUM_LAYERS = 2
NUM_LAYERS = 1

D_MODEL = 128
NUM_HEADS = 4
UNITS = 256
DROPOUT = 0.2


model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

In [57]:
# Learning rate
learning_rate = CustomSchedule(D_MODEL)

# Optimizer = Adam
optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  # ensure labels have shape (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [58]:
EPOCHS = 10

In [59]:
model.fit(dataset, epochs=EPOCHS)

Epoch 1/10
223/223 [==============================] - 71s 290ms/step - loss: 0.6404 - accuracy: 0.0121
Epoch 2/10
223/223 [==============================] - 65s 290ms/step - loss: 0.3795 - accuracy: 0.0509
Epoch 3/10
223/223 [==============================] - 65s 290ms/step - loss: 0.2151 - accuracy: 0.0598
Epoch 4/10
223/223 [==============================] - 65s 289ms/step - loss: 0.1515 - accuracy: 0.0673
Epoch 5/10
223/223 [==============================] - 64s 289ms/step - loss: 0.1136 - accuracy: 0.0715
Epoch 6/10
223/223 [==============================] - 65s 289ms/step - loss: 0.0898 - accuracy: 0.0749
Epoch 7/10
223/223 [==============================] - 65s 290ms/step - loss: 0.0759 - accuracy: 0.0768
Epoch 8/10
223/223 [==============================] - 65s 290ms/step - loss: 0.0679 - accuracy: 0.0778
Epoch 9/10
223/223 [==============================] - 65s 290ms/step - loss: 0.0632 - accuracy: 0.0784
Epoch 10/10
223/223 [==============================] - 64s 289ms/step - l

In [60]:
def evaluate(sentence):
  sentence = preprocess_sentence(sentence)

  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  for i in range(MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False)

    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)


def predict(sentence):
  prediction = evaluate(sentence)

  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  return predicted_sentence


def preprocess_sentence(sentence):
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = sentence.strip()
  return sentence

In [61]:
predict(filtering_train_df['split_label_token'][2])

'<esp> RED SPOTTY CHARLOTTE BAG <esp>'

In [62]:
filtering_test_df['split_train_token'] = ['<esp> ' + i.strip().replace('&&', ' <esp> ') + ' <esp>' for i in filtering_test_df['train']]
filtering_test_df['split_label_token'] = ['<esp> ' + i.strip() + ' <esp>' for i in filtering_test_df['label']]

In [64]:
filtering_test_df.head()

,train,label,split_train_token,split_label_token
0,DANISH ROSE PHOTO FRAME&&RETRO BLUE SPOTTY WAS...,WOODEN UNION JACK BUNTING,<esp> DANISH ROSE PHOTO FRAME <esp> RETRO BLUE...,<esp> WOODEN UNION JACK BUNTING <esp>
1,SET OF 2 ROUND TINS DUTCH CHEESE&&LARGE MEDINA...,LOVE BUILDING BLOCK WORD,<esp> SET OF 2 ROUND TINS DUTCH CHEESE <esp> L...,<esp> LOVE BUILDING BLOCK WORD <esp>
2,PINK 3 PIECE MINI DOTS CUTLERY SET&&CHILDS APR...,MINI JIGSAW SPACEBOY,<esp> PINK 3 PIECE MINI DOTS CUTLERY SET <esp>...,<esp> MINI JIGSAW SPACEBOY <esp>
3,WOODEN BOX OF DOMINOES&&TEA TIME DES TEA COSY&...,GREEN GINGHAM FLOWER JEWELLERY BOX,<esp> WOODEN BOX OF DOMINOES <esp> TEA TIME DE...,<esp> GREEN GINGHAM FLOWER JEWELLERY BOX <esp>
4,ASSORTED COLOURS SILK FAN&&PACK OF 72 SKULL CA...,WORLD WAR 2 GLIDERS ASSTD DESIGNS,<esp> ASSORTED COLOURS SILK FAN <esp> PACK OF ...,<esp> WORLD WAR 2 GLIDERS ASSTD DESIGNS <esp>


In [65]:
tqdm.tqdm.pandas()
predict_item_test = filtering_test_df['split_train_token'].progress_apply(predict)

100%|██████████| 14241/14241 [2:20:01<00:00,  1.70it/s]


In [66]:
filtering_test_df['predict_item'] = predict_item_test

In [67]:
filtering_test_df

,train,label,split_train_token,split_label_token,predict_item
0,DANISH ROSE PHOTO FRAME&&RETRO BLUE SPOTTY WAS...,WOODEN UNION JACK BUNTING,<esp> DANISH ROSE PHOTO FRAME <esp> RETRO BLUE...,<esp> WOODEN UNION JACK BUNTING <esp>,<esp> HOT BATHS METAL SIGN <esp>
1,SET OF 2 ROUND TINS DUTCH CHEESE&&LARGE MEDINA...,LOVE BUILDING BLOCK WORD,<esp> SET OF 2 ROUND TINS DUTCH CHEESE <esp> L...,<esp> LOVE BUILDING BLOCK WORD <esp>,<esp> RED SPOTTY APRON <esp>
2,PINK 3 PIECE MINI DOTS CUTLERY SET&&CHILDS APR...,MINI JIGSAW SPACEBOY,<esp> PINK 3 PIECE MINI DOTS CUTLERY SET <esp>...,<esp> MINI JIGSAW SPACEBOY <esp>,<esp> RED 3 PIECE MINI DOTS CUTLERY SET <esp>
3,WOODEN BOX OF DOMINOES&&TEA TIME DES TEA COSY&...,GREEN GINGHAM FLOWER JEWELLERY BOX,<esp> WOODEN BOX OF DOMINOES <esp> TEA TIME DE...,<esp> GREEN GINGHAM FLOWER JEWELLERY BOX <esp>,<esp> RED SPOTTY BISCUIT TIN <esp>
4,ASSORTED COLOURS SILK FAN&&PACK OF 72 SKULL CA...,WORLD WAR 2 GLIDERS ASSTD DESIGNS,<esp> ASSORTED COLOURS SILK FAN <esp> PACK OF ...,<esp> WORLD WAR 2 GLIDERS ASSTD DESIGNS <esp>,<esp> SET OF 72 RETRO SPOT PAPER DOILIES <esp>
...,...,...,...,...,...
14236,ASSORTED COLOUR BIRD ORNAMENT&&FILIGRIS HEART ...,HANGING WOOD AND FELT HEART,<esp> ASSORTED COLOUR BIRD ORNAMENT <esp> FILI...,<esp> HANGING WOOD AND FELT HEART <esp>,<esp> HANGING HEART ZINC T-LIGHT HOLDER <esp>
14237,LADLE LOVE HEART RED&&FAWN AND MUSHROOM GREETI...,SET/6 RED SPOTTY PAPER PLATES,<esp> LADLE LOVE HEART RED <esp> FAWN AND MUSH...,<esp> SET/6 RED SPOTTY PAPER PLATES <esp>,<esp> PACK OF 72 RETRO SPOT CAKE CASES <esp>
14238,GOLD CHERRY LIGHTS&&URBAN CHIC CHOPSTICKS SET...,WOBBLING METAL CHICKEN EASTER,<esp> GOLD CHERRY LIGHTS <esp> URBAN CHIC CHO...,<esp> WOBBLING METAL CHICKEN EASTER <esp>,<esp> PINK FAIRY CAKE CHILD'S APRON <esp>
14239,SET/2 RED SPOTTY TEA TOWELS&&ABSTRACT CIRCLES ...,HOME SWEET HOME BLACKBOARD,<esp> SET/2 RED SPOTTY TEA TOWELS <esp> ABSTRA...,<esp> HOME SWEET HOME BLACKBOARD <esp>,<esp> DOOR MAT UNION FLAG <esp>


# Product2Vec

In [68]:
TFoutputs = filtering_test_df

In [69]:
checker = pd.Series([i in detail for i in TFoutputs['predict_item']])
print(checker.value_counts())
filter = checker[checker == False].index
len(filter)

False    14241
dtype: int64


14241

In [70]:
TFoutputs['filter'] = checker
TFoutputs

,train,label,split_train_token,split_label_token,predict_item,filter
0,DANISH ROSE PHOTO FRAME&&RETRO BLUE SPOTTY WAS...,WOODEN UNION JACK BUNTING,<esp> DANISH ROSE PHOTO FRAME <esp> RETRO BLUE...,<esp> WOODEN UNION JACK BUNTING <esp>,<esp> HOT BATHS METAL SIGN <esp>,False
1,SET OF 2 ROUND TINS DUTCH CHEESE&&LARGE MEDINA...,LOVE BUILDING BLOCK WORD,<esp> SET OF 2 ROUND TINS DUTCH CHEESE <esp> L...,<esp> LOVE BUILDING BLOCK WORD <esp>,<esp> RED SPOTTY APRON <esp>,False
2,PINK 3 PIECE MINI DOTS CUTLERY SET&&CHILDS APR...,MINI JIGSAW SPACEBOY,<esp> PINK 3 PIECE MINI DOTS CUTLERY SET <esp>...,<esp> MINI JIGSAW SPACEBOY <esp>,<esp> RED 3 PIECE MINI DOTS CUTLERY SET <esp>,False
3,WOODEN BOX OF DOMINOES&&TEA TIME DES TEA COSY&...,GREEN GINGHAM FLOWER JEWELLERY BOX,<esp> WOODEN BOX OF DOMINOES <esp> TEA TIME DE...,<esp> GREEN GINGHAM FLOWER JEWELLERY BOX <esp>,<esp> RED SPOTTY BISCUIT TIN <esp>,False
4,ASSORTED COLOURS SILK FAN&&PACK OF 72 SKULL CA...,WORLD WAR 2 GLIDERS ASSTD DESIGNS,<esp> ASSORTED COLOURS SILK FAN <esp> PACK OF ...,<esp> WORLD WAR 2 GLIDERS ASSTD DESIGNS <esp>,<esp> SET OF 72 RETRO SPOT PAPER DOILIES <esp>,False
...,...,...,...,...,...,...
14236,ASSORTED COLOUR BIRD ORNAMENT&&FILIGRIS HEART ...,HANGING WOOD AND FELT HEART,<esp> ASSORTED COLOUR BIRD ORNAMENT <esp> FILI...,<esp> HANGING WOOD AND FELT HEART <esp>,<esp> HANGING HEART ZINC T-LIGHT HOLDER <esp>,False
14237,LADLE LOVE HEART RED&&FAWN AND MUSHROOM GREETI...,SET/6 RED SPOTTY PAPER PLATES,<esp> LADLE LOVE HEART RED <esp> FAWN AND MUSH...,<esp> SET/6 RED SPOTTY PAPER PLATES <esp>,<esp> PACK OF 72 RETRO SPOT CAKE CASES <esp>,False
14238,GOLD CHERRY LIGHTS&&URBAN CHIC CHOPSTICKS SET...,WOBBLING METAL CHICKEN EASTER,<esp> GOLD CHERRY LIGHTS <esp> URBAN CHIC CHO...,<esp> WOBBLING METAL CHICKEN EASTER <esp>,<esp> PINK FAIRY CAKE CHILD'S APRON <esp>,False
14239,SET/2 RED SPOTTY TEA TOWELS&&ABSTRACT CIRCLES ...,HOME SWEET HOME BLACKBOARD,<esp> SET/2 RED SPOTTY TEA TOWELS <esp> ABSTRA...,<esp> HOME SWEET HOME BLACKBOARD <esp>,<esp> DOOR MAT UNION FLAG <esp>,False


In [71]:
pred = TFoutputs['predict_item']
pred = pred.str.replace('<esp>', '')
pred = pred.str.strip()
pred.head()

0                   HOT BATHS METAL SIGN
1                       RED SPOTTY APRON
2      RED 3 PIECE MINI DOTS CUTLERY SET
3                 RED SPOTTY BISCUIT TIN
4    SET OF 72 RETRO SPOT PAPER  DOILIES
Name: predict_item, dtype: object

In [72]:
falses = pd.Series([TFoutputs['predict_item'][i] for i in filter])
falses = falses.str.replace('<esp>', '')
falses = falses.str.strip()
print(len(falses))
falses.head()

14241


0                   HOT BATHS METAL SIGN
1                       RED SPOTTY APRON
2      RED 3 PIECE MINI DOTS CUTLERY SET
3                 RED SPOTTY BISCUIT TIN
4    SET OF 72 RETRO SPOT PAPER  DOILIES
dtype: object

In [73]:
filter[:5]

Int64Index([0, 1, 2, 3, 4], dtype='int64')

In [74]:
df = df_new.copy()
df

,Detail,label
0,HAND OPEN SHAPE GOLD&&RETRO SPOT MUG&&RETRO SP...,RETRO SPOT TRADITIONAL TEAPOT
1,RETRO SPOT MUG&&RETRO SPOT SUGAR JAM BOWL&&NEW...,RETRO SPOT LARGE MILK JUG
2,MOROCCAN TEA GLASS&&BLACK FLOWER CANDLE PLATE&...,CERAMIC CAKE DESIGN SPOTTED PLATE
3,BLACK SILOUETTE CANDLE PLATE&&VICTORIAN GLASS ...,SAVE THE PLANET COTTON TOTE BAG
4,CERAMIC CAKE DESIGN SPOTTED PLATE&&PINK CLEAR ...,STRAWBERRY CERAMIC TRINKET BOX
...,...,...
71200,WOODEN SKITTLES GARDEN SET&&DOORMAT UNION FLAG...,WOODEN CROQUET GARDEN SET
71201,UNION STRIPE WITH FRINGE HAMMOCK&&RED/CREAM S...,WHITE HANGING HEART T-LIGHT HOLDER
71202,OCEAN STRIPE HAMMOCK&&BLUE/CREAM STRIPE FRINGE...,ENAMEL WATERING CAN CREAM
71203,WHITE HANGING HEART T-LIGHT HOLDER&&BAG 250g S...,NATURAL SLATE HEART CHALKBOARD


In [75]:
detail = detail.tolist()
len(detail)

3485

In [76]:
def JaccardSimilarity(inp1, inp2):
    list_inp1 = inp1.split()
    list_inp2 = inp2.split()
    mom = set(list_inp1).union(set(list_inp2))
    son = set(list_inp1).intersection(set(list_inp2))
    # print(mom)
    # print(son)
    return len(son)/len(mom)

In [77]:
pd.Series([i in detail for i in pred]).value_counts()

True     14222
False       19
dtype: int64

In [78]:
asdf = []

for v in falses:
    m = 0
    for d in detail:
        score = JaccardSimilarity(v, d)
        if m < score:
            m = score
            temp = d
    if m == 0: asdf.append(v)
    id = pred[pred == v].index
    for i in id:
        pred[i] = temp

len(asdf)

0

In [79]:
pd.Series([i in detail for i in pred]).value_counts()

True    14241
dtype: int64

In [80]:
# Word2Vec Model
model = Word2Vec([item_list.split('&&') for item_list in df['Detail']], size=100, window=5, min_count=1, negative=15, iter=20, sample=0.00001, sg=1)

In [81]:
def find_item(item, n=10):
    return list(dict(model.wv.most_similar(item, topn=n)))

In [82]:
find_item(pred[0])

['PINK MILKSHAKE GLASS',
 'BEACH HUT MIRROR',
 'SILVER/BLACK ORBIT NECKLACE',
 'CHERRY BLOSSOM PASSPORT COVER',
 'SILVER/M.O.P PENDANT ORBIT NECKLACE',
 'GREEN DROP EARRINGS W BEAD CLUSTER',
 'BAROQUE BUTTERFLY EARRINGS MONTANA',
 'DOORMAT SPOTTY HOME SWEET HOME',
 '5 STRAND GLASS NECKLACE AMBER',
 'METALIC LEAVES BAG CHARMS']

In [83]:
def recommend(n=10, acc=False, pred=pred, TFoutputs=TFoutputs):
    item_list = []
    for v in tqdm_notebook(pred.to_list()):
        if n > 1:
            pp = find_item(v, n-1)
            pp.insert(0, v)
            item_list.append(pp)
        else: item_list.append(v)
    TFoutputs['rec_list'] = item_list
    TFoutputs['is_in'] = [True if label in item else False for label, item in tqdm_notebook(zip(TFoutputs['label'], TFoutputs['rec_list']), total=(len(TFoutputs)))]

    if acc:
        return TFoutputs[TFoutputs['is_in']==True].shape[0]/(TFoutputs.shape[0])

    return TFoutputs

In [84]:
recommend(n=5, acc=True)

  0%|          | 0/14241 [00:00<?, ?it/s]

  0%|          | 0/14241 [00:00<?, ?it/s]

0.01734428762025139

In [85]:
recommend(n=10, acc=True)

  0%|          | 0/14241 [00:00<?, ?it/s]

  0%|          | 0/14241 [00:00<?, ?it/s]

0.01769538655993259

In [86]:
recommend(n=20, acc=True)

  0%|          | 0/14241 [00:00<?, ?it/s]

  0%|          | 0/14241 [00:00<?, ?it/s]

0.019099782318657398

In [87]:
recommend()

  0%|          | 0/14241 [00:00<?, ?it/s]

  0%|          | 0/14241 [00:00<?, ?it/s]

,train,label,split_train_token,split_label_token,predict_item,filter,rec_list,is_in
0,DANISH ROSE PHOTO FRAME&&RETRO BLUE SPOTTY WAS...,WOODEN UNION JACK BUNTING,<esp> DANISH ROSE PHOTO FRAME <esp> RETRO BLUE...,<esp> WOODEN UNION JACK BUNTING <esp>,<esp> HOT BATHS METAL SIGN <esp>,False,"[HOT BATHS METAL SIGN, PINK MILKSHAKE GLASS, B...",False
1,SET OF 2 ROUND TINS DUTCH CHEESE&&LARGE MEDINA...,LOVE BUILDING BLOCK WORD,<esp> SET OF 2 ROUND TINS DUTCH CHEESE <esp> L...,<esp> LOVE BUILDING BLOCK WORD <esp>,<esp> RED SPOTTY APRON <esp>,False,"[RED SPOTTY APRON, GREEN DROP EARRINGS W BEAD ...",False
2,PINK 3 PIECE MINI DOTS CUTLERY SET&&CHILDS APR...,MINI JIGSAW SPACEBOY,<esp> PINK 3 PIECE MINI DOTS CUTLERY SET <esp>...,<esp> MINI JIGSAW SPACEBOY <esp>,<esp> RED 3 PIECE MINI DOTS CUTLERY SET <esp>,False,"[RED 3 PIECE MINI DOTS CUTLERY SET, PINK STITC...",False
3,WOODEN BOX OF DOMINOES&&TEA TIME DES TEA COSY&...,GREEN GINGHAM FLOWER JEWELLERY BOX,<esp> WOODEN BOX OF DOMINOES <esp> TEA TIME DE...,<esp> GREEN GINGHAM FLOWER JEWELLERY BOX <esp>,<esp> RED SPOTTY BISCUIT TIN <esp>,False,"[RED SPOTTY BISCUIT TIN, COPPER/OLIVE GREEN FL...",False
4,ASSORTED COLOURS SILK FAN&&PACK OF 72 SKULL CA...,WORLD WAR 2 GLIDERS ASSTD DESIGNS,<esp> ASSORTED COLOURS SILK FAN <esp> PACK OF ...,<esp> WORLD WAR 2 GLIDERS ASSTD DESIGNS <esp>,<esp> SET OF 72 RETRO SPOT PAPER DOILIES <esp>,False,"[SET OF 72 RETRO SPOT PAPER DOILIES, SMALL CH...",False
...,...,...,...,...,...,...,...,...
14236,ASSORTED COLOUR BIRD ORNAMENT&&FILIGRIS HEART ...,HANGING WOOD AND FELT HEART,<esp> ASSORTED COLOUR BIRD ORNAMENT <esp> FILI...,<esp> HANGING WOOD AND FELT HEART <esp>,<esp> HANGING HEART ZINC T-LIGHT HOLDER <esp>,False,"[HANGING HEART ZINC T-LIGHT HOLDER, DIAMANTE B...",False
14237,LADLE LOVE HEART RED&&FAWN AND MUSHROOM GREETI...,SET/6 RED SPOTTY PAPER PLATES,<esp> LADLE LOVE HEART RED <esp> FAWN AND MUSH...,<esp> SET/6 RED SPOTTY PAPER PLATES <esp>,<esp> PACK OF 72 RETRO SPOT CAKE CASES <esp>,False,"[PACK OF 72 RETRO SPOT CAKE CASES, FRESHWATER ...",False
14238,GOLD CHERRY LIGHTS&&URBAN CHIC CHOPSTICKS SET...,WOBBLING METAL CHICKEN EASTER,<esp> GOLD CHERRY LIGHTS <esp> URBAN CHIC CHO...,<esp> WOBBLING METAL CHICKEN EASTER <esp>,<esp> PINK FAIRY CAKE CHILD'S APRON <esp>,False,"[PINK FAIRY CAKE CHILD'S APRON, FIRE POLISHED ...",False
14239,SET/2 RED SPOTTY TEA TOWELS&&ABSTRACT CIRCLES ...,HOME SWEET HOME BLACKBOARD,<esp> SET/2 RED SPOTTY TEA TOWELS <esp> ABSTRA...,<esp> HOME SWEET HOME BLACKBOARD <esp>,<esp> DOOR MAT UNION FLAG <esp>,False,"[DOOR MAT UNION FLAG, PEARL & SHELL 42""NECKL. ...",False
